In [1]:
#general packages
import pandas as pd
from itertools import product
from barcode_key_converter import *

# This notebook is used to convert the old barcode key.

In [2]:
#read in codebook
codebook = pd.read_csv("../488nm_codebook.csv", index_col = 0)

In [3]:
#keep only normal genes (old codebook generation may have missing fake codes)
blanks = codebook[codebook.index.str.startswith("blank")].index
codebook = codebook.drop(blanks)

In [4]:
#generate all possible codewords
#range of all pseudocolors
ps = np.arange(1,16,1)
#generate all possible combinations
number_of_sites = 3
perm_ps = list(product(ps,repeat=number_of_sites))

In [5]:
#make sure the length is equal pseudocolor^number of sites
len(perm_ps) == 15**3

True

In [6]:
#make list of codes that has been used
codes = codebook[["hyb1","hyb2","hyb3"]].values
code_set = [tuple(i) for i in codes]

In [7]:
#find unused codewords
diff = list(set(perm_ps) - set(code_set))
len(diff)

375

In [ ]:
#make fakebook
fakecodes = pd.DataFrame(diff)
fakecodes.columns = ["hyb1","hyb2","hyb3"]

# This cell block is if you need to add parity codes.

In [8]:
#get fake code with error correction (sum(total readout sites -1)) mod total pseudocolor)
errorfakecode = []
total_pseudocolors = 15
for code in diff:
    error = (code[0]+code[1]-code[2]) % total_pseudocolors
    if error == 0:
        error = total_pseudocolors
    errorfakecode.append(int(error))
    
fakecodes["hyb4"] = errorfakecode

--------------------------------------------------------------------------------------------------------------------

In [10]:
#make fake code names
fakegenes = []
for i in range(len(fakecodes)):
    fakegenes.append(f"fake{i}")

In [11]:
#add fake names
fakecodes.index = fakegenes

In [12]:
#add to codebook
codebook = pd.concat([codebook,fakecodes])

In [13]:
codebook

,hyb1,hyb2,hyb3,hyb4
auts2,6,14,5,15
meis2,5,11,6,10
ptprd,1,6,7,15
lsamp,1,15,10,6
robo1,4,11,12,3
dach1,15,12,1,11
prkg1,8,1,13,11
ext1,7,4,11,15
aff3,4,7,15,11
2610307p16rik,4,4,7,1


In [15]:
#generate new codebook
new_codebook = barcode_key_converter_within(codebook, num_hybs = 60, num_barcodes = 4, ch=2)

In [16]:
#take a look to see it makes sense
new_codebook.iloc[0].values

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2])

In [17]:
#write out new codebook
new_codebook.to_csv("codebook_string_488.csv")